Paso 1: Instalar las bibliotecas necesarias desde la terminal o celda

In [ ]:
!pip install SQLAlchemy==2.0.31
!pip install langchain==0.0.292
!pip install tiktoken==0.5.1
!pip install docarray==0.38.0
!pip install openai==0.28
!pip install yt_dlp
!pip install youtube-dl
!pip install pydub
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.4
    Uninstalling pydantic-2.7.4:
      Successfully uninstalled pydantic-2.7.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Paso 3: Importar los paquetes necesarios

In [ ]:
import os
import glob
import openai
import yt_dlp as youtube_dl
from yt_dlp import DownloadError
import docarray
import pydub
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown


Paso 4: Configurar la clave de API de OpenAI (revisar por variable de entorno)

In [ ]:
openai.api_key = "sk-proj-NCeH7elzvrTgjL2rPQJrT3BlbkFJ8MQDCZWYC2inGXbiWaPS"


Paso 5: Descargar video de YouTube y extraer audio en mp3

In [ ]:
# Select the video url
youtube_url = "https://www.youtube.com/watch?v=CTl3qVdfVT8"

# Point the Directory to store the downloaded video
output_dir = "content/audio/"

# Config for youtube-dl
ydl_config = {
    "format": "bestaudio/best",
    "postprocessors": [
        {
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }
    ],
    "outtmpl": os.path.join(output_dir, "%(title)s.%(ext)s"),
    "verbose": True
}

# Check if the output directory exists, otherwise create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Print a message indicating which video is being downloaded
print(f"Downloading video from {youtube_url}")

try:
    # Download the video using youtube_dl
    with youtube_dl.YoutubeDL(ydl_config) as ydl:
        ydl.download([youtube_url])

    # Check if the downloaded file is a WebM
    downloaded_file = os.path.join(output_dir, ydl.extract_info(youtube_url, download=False)['title'] + ".webm")
    if os.path.exists(downloaded_file):
        # Extract audio from WebM file using pydub
        sound = pydub.AudioSegment.from_file(downloaded_file, format="webm")
        output_filename = f"{output_dir}/{ydl.extract_info(youtube_url, download=False)['title']}.mp3"
        sound.export(output_filename, format="mp3", bitrate="192k")  # Adjust bitrate if needed

        os.remove(downloaded_file)  # Remove the WebM file after successful extraction
    else:
        print(f"Downloaded audio: {downloaded_file}")

except DownloadError as e:
    # Download error occurred, handle it (e.g., print error message or retry logic)
    print(f"Download failed: {e}")


[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.05.27 from yt-dlp/yt-dlp [12b248ce6] (pip) API
[debug] params: {'format': 'bestaudio/best', 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}], 'outtmpl': 'content/audio/%(title)s.%(ext)s', 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.85+-x86_64-with-glibc2.35 (OpenSSL 3.0.2 15 Mar 2022, glibc 2.35)


[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: Cryptodome-3.20.0, brotli-1.1.0, certifi-2024.06.02, mutagen-1.47.0, requests-2.31.0, secretstorage-3.3.1, sqlite3-3.37.2, urllib3-2.0.7, websockets-12.0
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}
[debug] Request Handlers: urllib, requests, websockets
[debug] Loaded 1820 extractors


[youtube] Extracting URL: https://www.youtube.com/watch?v=CTl3qVdfVT8
[youtube] CTl3qVdfVT8: Downloading webpage
[youtube] CTl3qVdfVT8: Downloading ios player API JSON
[youtube] CTl3qVdfVT8: Downloading player 5b22937f


[debug] [youtube] Extracting nsig function with jsinterp
[debug] Saving youtube-nsig.5b22937f to cache
[debug] [youtube] Decrypted nsig mlw4AYbLQvca5hrj => w7embO5YcnGPRQ
[debug] Loading youtube-nsig.5b22937f from cache
[debug] [youtube] Decrypted nsig FYNPQTttwUDoOQ8H => SsJFAP5sMap_pQ


[youtube] CTl3qVdfVT8: Downloading m3u8 information


...

Deleting original file content/audio/iPhone 15 Review： LA DECISIÓN MÁS INTELIGENTE.webm (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=CTl3qVdfVT8
[youtube] CTl3qVdfVT8: Downloading webpage
[youtube] CTl3qVdfVT8: Downloading ios player API JSON


[debug] Loading youtube-nsig.5b22937f from cache
[debug] [youtube] Decrypted nsig XAJOAY6RaA4pcUyZ => Zu2hVcVgFrrfuQ
[debug] Loading youtube-nsig.5b22937f from cache
[debug] [youtube] Decrypted nsig BlLb9fl9ThLxELhV => VRDfGziNPQ8IDw


[youtube] CTl3qVdfVT8: Downloading m3u8 information


[debug] Sort order given by extractor: quality, res, fps, hdr:12, source, vcodec:vp9.2, channels, acodec, lang, proto
[debug] Formats sorted by: hasvid, ie_pref, quality, res, fps, hdr:12(7), source, vcodec:vp9.2(10), channels, acodec, lang, proto, size, br, asr, vext, aext, hasaud, id


Downloaded audio: content/audio/iPhone 15 Review: LA DECISIÓN MÁS INTELIGENTE.webm


Paso 6: Encontrar archivos de audio descargado

In [ ]:
# Find all the audio files in the output directory
audio_files = glob.glob(os.path.join(output_dir, "*.mp3"))

# Check if any audio files were found
if audio_files:
  # Select the first audio file in the list
  audio_filename = audio_files[0]
  # Print the name of the selected audio file
  print(audio_filename)
else:
  print("No audio files found in the output directory.")


content/audio/iPhone 15 Review： LA DECISIÓN MÁS INTELIGENTE.mp3


Paso 7: Transcribir el audio utilizando Whisper

In [ ]:
# Transcribe the audio file to text using OpenAI API
audio_file = audio_filename
output_file = "files/transcripts/transcript.txt"
model = "whisper-1"

print("Converting audio to text...")

with open(audio_file, "rb") as audio:
    response = openai.Audio.transcribe(model, audio)

# Extract the transcript from the response
transcript = response["text"]

# If an output file is specified, save the transcript to a .txt file
if output_file is not None:
    # Create the directory for the output file if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Write the transcript to the output file
    with open(output_file, "w") as file:
        file.write(transcript)

# Print the transcript to the console to verify if it worked
print(transcript)


Converting audio to text...
He estado probando el nuevo iPhone 15 durante bastantes días como para poder traeros este análisis y lo primero de todo que os voy a decir es que me parece un modelo muchísimo más interesante que el iPhone 15 Pro o que el iPhone 15 Pro Max. En este vídeo te voy a decir que este iPhone 15 probablemente sea el iPhone que estabas esperando desde hace tiempo... ¿o no? Muy buenas ataqueros, ¿qué tal estáis? Yo soy Mario Arroyo y por primera vez en varias generaciones de iPhone al fin podemos diferenciar a simple vista el iPhone 15 del iPhone 14. Esto se debe principalmente a que Apple ha integrado también la Dynamic Island o isla dinámica en estos modelos básicos y bueno este es un elemento que tiene ciertas ventajas a la hora de usar el teléfono pero yo después de un año utilizando el iPhone 14 Pro Max como mi teléfono personal tengo que decir que tampoco le he sacado muchísimo partido y creo que todavía le falta más integración de aplicaciones de terceros. Habl

Paso 8: Crear un TextLoader utilizando LangChain

In [ ]:
# Initialize the TextLoader with the transcript
loader = TextLoader('files/transcripts/transcript.txt')

# Load the documents using the loader
documents = loader.load()

# Display the first document to verify if it was loaded correctly
display(documents[0])


Document(page_content='He estado probando el nuevo iPhone 15 durante bastantes días como para poder traeros este análisis y lo primero de todo que os voy a decir es que me parece un modelo muchísimo más interesante que el iPhone 15 Pro o que el iPhone 15 Pro Max. En este vídeo te voy a decir que este iPhone 15 probablemente sea el iPhone que estabas esperando desde hace tiempo... ¿o no? Muy buenas ataqueros, ¿qué tal estáis? Yo soy Mario Arroyo y por primera vez en varias generaciones de iPhone al fin podemos diferenciar a simple vista el iPhone 15 del iPhone 14. Esto se debe principalmente a que Apple ha integrado también la Dynamic Island o isla dinámica en estos modelos básicos y bueno este es un elemento que tiene ciertas ventajas a la hora de usar el teléfono pero yo después de un año utilizando el iPhone 14 Pro Max como mi teléfono personal tengo que decir que tampoco le he sacado muchísimo partido y creo que todavía le falta más integración de aplicaciones de terceros. Hablando 

Paso 9: Dividir el texto en fragmentos utilizando el CharacterTextSplitter

In [ ]:
# Initialize the CharacterTextSplitter with desired settings
text_splitter = CharacterTextSplitter(
    chunk_size=1000,  # Maximum size of each chunk
    chunk_overlap=200  # Overlap between chunks
)

# Split the text into chunks using the text splitter
texts = text_splitter.split_documents(documents)

# Display the first chunk to verify if it was split correctly
display(texts[0])


Document(page_content='He estado probando el nuevo iPhone 15 durante bastantes días como para poder traeros este análisis y lo primero de todo que os voy a decir es que me parece un modelo muchísimo más interesante que el iPhone 15 Pro o que el iPhone 15 Pro Max. En este vídeo te voy a decir que este iPhone 15 probablemente sea el iPhone que estabas esperando desde hace tiempo... ¿o no? Muy buenas ataqueros, ¿qué tal estáis? Yo soy Mario Arroyo y por primera vez en varias generaciones de iPhone al fin podemos diferenciar a simple vista el iPhone 15 del iPhone 14. Esto se debe principalmente a que Apple ha integrado también la Dynamic Island o isla dinámica en estos modelos básicos y bueno este es un elemento que tiene ciertas ventajas a la hora de usar el teléfono pero yo después de un año utilizando el iPhone 14 Pro Max como mi teléfono personal tengo que decir que tampoco le he sacado muchísimo partido y creo que todavía le falta más integración de aplicaciones de terceros. Hablando 

Paso 10: Crear un índice FAISS a partir de los fragmentos de texto

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 44.4 MB/s eta 0:00:00


In [ ]:
# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

# Create a FAISS index from the text chunks
faiss_index = FAISS.from_texts([text.page_content for text in texts], embeddings)

# Display a message to indicate that the index was created successfully
print("FAISS index created successfully.")


FAISS index created successfully.


Paso 11: Crear una cadena de recuperación de preguntas y respuestas (RetrievalQA) 👉 Subir la variable de entorno para contraseña al inicio del programa

In [ ]:
# Initialize the OpenAI chat model
import openai
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-NCeH7elzvrTgjL2rPQJrT3BlbkFJ8MQDCZWYC2inGXbiWaPS"  # Set API key as environment variable
chat_model = ChatOpenAI(model_name="gpt-4")

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=faiss_index.as_retriever()
)

# Display a message to indicate that the RetrievalQA chain was created successfully
print("RetrievalQA chain created successfully.")

RetrievalQA chain created successfully.


Paso 12: Preguntar al modelo para probar el retrieval de información

In [ ]:
# Ask a sample question using the QA chain
question = "What is the main topic of the video?"
response = qa_chain.run(question)

# Display the response
print(f"Question: {question}")
print(f"Answer: {response}")


Question: What is the main topic of the video?
Answer: The main topic of the video is a review of the new iPhone 15. The speaker provides a detailed analysis of the phone's features, design, performance, camera quality, and battery life, comparing it to other models like the iPhone 15 Pro and iPhone 15 Pro Max. The speaker expresses their preference for the iPhone 15 over its Pro counterparts.


In [ ]:
# Set the query to be user for the QA system
question = "Who should what this video?"

# Run the question through the RetrievalQA instance and the response

response = qa_chain.run(question)

# Print the response to the console

print(response)

This video should be watched by anyone interested in getting an in-depth review of the iPhone 15 and iPhone 15 Plus, particularly those who are considering purchasing one of these models and want to understand their features, advantages and disadvantages.


Testing question not related with transcript (particular video)


In [ ]:
# Set the query to be user for the QA system
question = "Who is the brest football player in the world?"

# Run the question through the RetrievalQA instance and the response

response = qa_chain.run(question)

# Print the response to the console

print(response)

I'm sorry, but the text provided doesn't include information about the best football player in the world.


In [ ]:
# Set the query to be user for the QA system
question = "How long is the circumference of the earth?"

# Run the question through the RetrievalQA instance and the response

response = qa_chain.run(question)

# Print the response to the console

print(response)

I don't know.
